# RECHERCHE D'INFORMATION - TLN

## Etape 1 — Construction d’un corpus de documents sur le Covid-19  
Vous  devez  sauvegarder  le  contenu  textuel  present  dans  les  pages  web  suivantes  (un 
document txt pour chaque page web) : 
1. https://www.nature.com/articles/d41586-020-00502-w 
2. https://www.nejm.org/doi/full/10.1056/NEJMoa2033700?query-featured_coronavirus= 
3. https://www.nejm.org/doi/full/10.1056/NEJMoa2030340?query=featured_coronavirus 
4. https://www.nejm.org/doi/full/10.1056/NEJMoa2035002?query-featured_coronavirus= 
5. https://www.nejm.org/doi/full/10.1056/NEJMoa2029849?query=featured_coronavirus 
6. https://www.nejm.org/doi/full/10.1056/NEJMpv2035416?query=featured_coronavirus 
7. https://www.thelancet.com/journals/lanrhe/article/PIIS2665-9913(21)00007-2/fulltext 
8. https://www.thelancet.com/journals/lanres/article/PIIS2213-2600(21)00025-4/fulltext 
9. https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)32656-8/fulltext 
10. https://science.sciencemag.org/content/early/2021/01/11/science.abe6522 

* Les légendes des images ne doivent pas être sauvegardées. 
Le  résultat  de  cette  premiere  étape  est  un  corpus  de  10  documents  en  anglais  sur  le  
sujet de la Covid-19. 

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

liste_documents = [
    "https://www.nature.com/articles/d41586-020-00502-w",
    "https://www.nejm.org/doi/full/10.1056/NEJMoa2033700?query-featured_coronavirus=",
    "https://www.nejm.org/doi/full/10.1056/NEJMoa2030340?query=featured_coronavirus",
    "https://www.nejm.org/doi/full/10.1056/NEJMoa2035002?query-featured_coronavirus=",
    "https://www.nejm.org/doi/full/10.1056/NEJMoa2029849?query=featured_coronavirus",
    "https://www.nejm.org/doi/full/10.1056/NEJMpv2035416?query=featured_coronavirus",
    "https://www.thelancet.com/journals/lanrhe/article/PIIS2665-9913(21)00007-2/fulltext",
    "https://www.thelancet.com/journals/lanres/article/PIIS2213-2600(21)00025-4/fulltext",
    "https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)32656-8/fulltext",
    "https://science.sciencemag.org/content/early/2021/01/11/science.abe6522"]

files = ["Link1.txt", "Link2.txt", "Link3.txt","Link4.txt","Link5.txt","Link6.txt","Link7.txt","Link8.txt","Link9.txt","Link10.txt"]

nejmOrg = [1,2,3,4,5]
thelancet = [6,7,8,9]

for i in range(len(liste_documents)):
    # read 
    url = liste_documents[i]

    if i == 0 :
        page = urlopen(url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text()
        text = text.replace("\n", "")
        text = text.replace("  ", "")
        text = text.replace("―", "")
        text = str(text).encode("utf-8")

    if i in nejmOrg:
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')   
        doc_text = ""
        for p in soup.body.find_all('p', class_="f-body"):
            doc_text+=p.text.strip()
        text = str(doc_text).encode("utf-8")

    if i in thelancet : 
        page = requests.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        text = soup.get_text()
        text = text.replace("\n", "")
        text = str(text).encode("utf-8")

    
    names_file = files[i]
    file = open(names_file, "w") 
    
    #write
    file.write(str(text))
    file.close()



## Etape 2 — Pre-traitement des données et construction de la matrice d’incidence  

L’étape 2 est composée de 5 sous-taches séparées. Vous devez : 
 1. segmenter chaque document a l’aide d’un tokenizer et extraire les mots (c’est-à-dire couper la séquence de caractères en tokens/segments et les extraire). 
 2. éliminer les mots-vides (c’est-à-dire enlever les mots très courants, non porteurs de sens). 
 3. normaliser le vocabulaire des termes a travers la lemmatisation (c’est-à-dire mapper les mots sur les formes du diction 
 4. définir le dictionnaire relatif à cette collection de documents. 
 5. vous construisez la matrice d'incidence documents-ter
 Matrice d’incidence
•Idée : créer une structure compacte sur laquelle on va pouvoir effectuer des 
opérations de recherche.  
•Cette structure est une matrice dans laquelle on représente la présence ou 
non d’un mot dans un document.  
•On appelle cela la matrice d’incidence

#### Chargement 
Je cré ma dataframe avec deux colonnes : 
* une qui contient le texte du document chargé
* l'autre contient le nom du fichier 

In [1]:
import pandas as pd
files = ["Link1.txt", "Link2.txt", "Link3.txt","Link4.txt","Link5.txt","Link6.txt","Link7.txt","Link8.txt","Link9.txt","Link10.txt"]

documents = pd.DataFrame(columns=['Text', 'Document'])
for file in files : 
    f = open(file)
    raw = f.read()
    documents = documents.append({'Text': raw, 'Document':file }, ignore_index=True)

documents

,Text,Document
0,"b""COVID research: a year of scientific milesto...",Link1.txt
1,b'Therapies to interrupt the progression of ea...,Link2.txt
2,b'Coronavirus disease 2019 (Covid-19) pneumoni...,Link3.txt
3,b'Recent data suggest that complications and d...,Link4.txt
4,b'Severe acute respiratory syndrome coronaviru...,Link5.txt
5,b'Covid-19 has devastated refugees and asylum ...,Link6.txt
6,"b""COVID-19 and systemic sclerosis: clinicopath...",Link7.txt
7,"b""Improving family access to dying patients du...",Link8.txt
8,"b""6-month consequences of COVID-19 in patients...",Link9.txt
9,b' Immunological characteristics govern the tr...,Link10.txt


### 1. Segmenter chaque document à l'aide d'un tokenizer et extraire les mots
* J'ajoute une colonne des mots sous forme de token grâce à nlp 

In [2]:
import en_core_web_sm

import re
nlp = en_core_web_sm.load()

documents.insert(2,'Tokenize_Text',"")


for i in range(len(documents)):
    documents['Text'][i] = documents['Text'][i][1:]
    documents['Text'][i] = re.sub(r'[^\w\s]','',documents['Text'][i])
    documents['Tokenize_Text'][i] = nlp(documents['Text'][i])

documents

,Text,Document,Tokenize_Text
0,COVID research a year of scientific milestones...,Link1.txt,"(COVID, research, a, year, of, scientific, mil..."
1,Therapies to interrupt the progression of earl...,Link2.txt,"(Therapies, to, interrupt, the, progression, o..."
2,Coronavirus disease 2019 Covid19 pneumonia is ...,Link3.txt,"(Coronavirus, disease, 2019, Covid19, pneumoni..."
3,Recent data suggest that complications and dea...,Link4.txt,"(Recent, data, suggest, that, complications, a..."
4,Severe acute respiratory syndrome coronavirus ...,Link5.txt,"(Severe, acute, respiratory, syndrome, coronav..."
5,Covid19 has devastated refugees and asylum see...,Link6.txt,"(Covid19, has, devastated, refugees, and, asyl..."
6,COVID19 and systemic sclerosis clinicopatholog...,Link7.txt,"(COVID19, and, systemic, sclerosis, clinicopat..."
7,Improving family access to dying patients duri...,Link8.txt,"(Improving, family, access, to, dying, patient..."
8,6month consequences of COVID19 in patients dis...,Link9.txt,"(6month, consequences, of, COVID19, in, patien..."
9,Immunological characteristics govern the tran...,Link10.txt,"( , Immunological, characteristics, govern, th..."


### 2. Eliminer les mots-vides, c'est à dire enlerver les mots très courant, non porteurs de sens

In [3]:
from nltk.corpus import stopwords
documents.insert(3,'Tokenize_Text_Stop_word',"")

* Je retire les stopwords en anglais, et cré une nouvelle colonne de token sans les stop words

In [4]:
for i in range(len(documents)):
    token = []
    for text in  documents['Tokenize_Text'][i] :
        if text.text not in stopwords.words('english'):
            token.append(text)
        documents['Tokenize_Text_Stop_word'][i] = token

documents

,Text,Document,Tokenize_Text,Tokenize_Text_Stop_word
0,COVID research a year of scientific milestones...,Link1.txt,"(COVID, research, a, year, of, scientific, mil...","[COVID, research, year, scientific, milestones..."
1,Therapies to interrupt the progression of earl...,Link2.txt,"(Therapies, to, interrupt, the, progression, o...","[Therapies, interrupt, progression, early, cor..."
2,Coronavirus disease 2019 Covid19 pneumonia is ...,Link3.txt,"(Coronavirus, disease, 2019, Covid19, pneumoni...","[Coronavirus, disease, 2019, Covid19, pneumoni..."
3,Recent data suggest that complications and dea...,Link4.txt,"(Recent, data, suggest, that, complications, a...","[Recent, data, suggest, complications, death, ..."
4,Severe acute respiratory syndrome coronavirus ...,Link5.txt,"(Severe, acute, respiratory, syndrome, coronav...","[Severe, acute, respiratory, syndrome, coronav..."
5,Covid19 has devastated refugees and asylum see...,Link6.txt,"(Covid19, has, devastated, refugees, and, asyl...","[Covid19, devastated, refugees, asylum, seeker..."
6,COVID19 and systemic sclerosis clinicopatholog...,Link7.txt,"(COVID19, and, systemic, sclerosis, clinicopat...","[COVID19, systemic, sclerosis, clinicopatholog..."
7,Improving family access to dying patients duri...,Link8.txt,"(Improving, family, access, to, dying, patient...","[Improving, family, access, dying, patients, C..."
8,6month consequences of COVID19 in patients dis...,Link9.txt,"(6month, consequences, of, COVID19, in, patien...","[6month, consequences, COVID19, patients, disc..."
9,Immunological characteristics govern the tran...,Link10.txt,"( , Immunological, characteristics, govern, th...","[ , Immunological, characteristics, govern, tr..."


### 3. Normaliser le vocabulaire des termes à travers la lemmatisation 

In [5]:
documents.insert(4,'Lemma',"")

* J'ajoute une colonne qui prend les mots sans les stops word, et prend les lemme de ces mots 

In [6]:
for i in range(len(documents)):
    lemma = []
    for word in range(len(documents['Tokenize_Text_Stop_word'][i])) :
        lem = documents['Tokenize_Text_Stop_word'][i][word].lemma_
        lemma.append(lem)
    documents['Lemma'][i] = lemma

In [7]:
documents.head(3)

,Text,Document,Tokenize_Text,Tokenize_Text_Stop_word,Lemma
0,COVID research a year of scientific milestones...,Link1.txt,"(COVID, research, a, year, of, scientific, mil...","[COVID, research, year, scientific, milestones...","[COVID, research, year, scientific, milestones..."
1,Therapies to interrupt the progression of earl...,Link2.txt,"(Therapies, to, interrupt, the, progression, o...","[Therapies, interrupt, progression, early, cor...","[therapy, interrupt, progression, early, coron..."
2,Coronavirus disease 2019 Covid19 pneumonia is ...,Link3.txt,"(Coronavirus, disease, 2019, Covid19, pneumoni...","[Coronavirus, disease, 2019, Covid19, pneumoni...","[Coronavirus, disease, 2019, Covid19, pneumoni..."


### 4. Définir le dictionnaire relatif à cette collection de documents

In [8]:
documents.insert(5,'unique_word',"")

* Ici (au cas où) j'ai choisi d'ajouter une colonne avec les lemmes sans doublons 

In [9]:
import numpy as np

for i in range(len(documents)):
    documents["unique_word"][i]= np.unique(str(documents["Lemma"][i]))

documents.head(3)

,Text,Document,Tokenize_Text,Tokenize_Text_Stop_word,Lemma,unique_word
0,COVID research a year of scientific milestones...,Link1.txt,"(COVID, research, a, year, of, scientific, mil...","[COVID, research, year, scientific, milestones...","[COVID, research, year, scientific, milestones...","[['COVID', 'research', 'year', 'scientific', '..."
1,Therapies to interrupt the progression of earl...,Link2.txt,"(Therapies, to, interrupt, the, progression, o...","[Therapies, interrupt, progression, early, cor...","[therapy, interrupt, progression, early, coron...","[['therapy', 'interrupt', 'progression', 'earl..."
2,Coronavirus disease 2019 Covid19 pneumonia is ...,Link3.txt,"(Coronavirus, disease, 2019, Covid19, pneumoni...","[Coronavirus, disease, 2019, Covid19, pneumoni...","[Coronavirus, disease, 2019, Covid19, pneumoni...","[['Coronavirus', 'disease', '2019', 'Covid19',..."


### Création du dictionnaire 
* Contient le term, le doc dans lequel il se trouve et une colonne nb de fois qui pour le moment est rempli de 0

In [15]:
Dictionnaire = pd.DataFrame( columns=["Term", "Doc n°","nb_fois"])

for i in range(len(documents)):
    texte = documents["Lemma"][i]

    for mot in texte :
        Dictionnaire = Dictionnaire.append({"Term": mot,"Doc n°":i , "nb_fois":0}, ignore_index=True)

Dictionnaire

,Term,Doc n°,nb_fois
0,COVID,0,0
1,research,0,0
2,year,0,0
3,scientific,0,0
4,milestonesSkip,0,0
...,...,...,...
57558,ISSN,9,0
57559,00368075back,9,0
57560,topterm,9,0
57561,ServicePrivacy,9,0


* On retire les doublons, car des mots sont présent plusieurs fois dans un document, on enlève donc les lignes dont le term et le doc sont identiques

In [16]:
Dictionnaire = Dictionnaire.drop_duplicates(subset=None, 
                          keep='first', 
                          inplace=False, 
                          ignore_index=True) # on retire les doublons 
# c'est à dire, les lignes dont le mot et le doc sont similaire 
# (on a pas encore compté les occurences donc elles sont toutes à 0)

### Comptage du nombre d'occurence de chaque mot dans chaque doc
* Cela pourra nous être utile

In [18]:
# On peut commencer à compter les occurrences 
for i in range(len(Dictionnaire)): # pour chaque mot et document associé
    doc = Dictionnaire['Doc n°'][i]
    Term =  Dictionnaire['Term'][i]
    for j in range(len(documents)): # pour chaque document correspondant à la ligne du dict
        if j == doc: 
            for word in documents["Lemma"][j]:
                if word == Term: 
                    Dictionnaire.at[i,'nb_fois'] +=1
            
            
Dictionnaire

,Term,Doc n°,nb_fois
0,COVID,0,35
1,research,0,19
2,year,0,14
3,scientific,0,2
4,milestonesSkip,0,1
...,...,...,...
16954,COUNTER,9,1
16955,ISSN,9,1
16956,00368075back,9,1
16957,topterm,9,1


### Association des termes et document 
* Dictionnaire final contenant : 
    - Le term
    - Le doc dans lequel il se trouve
    - Le nombre de fois qu'il y apparaît
    - **Le nombre de document dans lequel il apparaît**

In [ ]:
Dictionnaire.insert(3,'nb_de_Doc',"")

In [63]:
# Ajout de la colonne qui compte le nombre de doc dans lesquels le terme apparait
for i in range(len(Dictionnaire)):
    term = Dictionnaire['Term'][i]
    lign = Dictionnaire[Dictionnaire['Term']== term].index
    Dictionnaire.at[i, 'nb_de_Doc'] = len(lign)

Dictionnaire

,Term,Doc n°,nb_fois,nb_de_Doc
0,COVID,0,35,4
1,research,0,19,4
2,year,0,14,8
3,scientific,0,2,1
4,milestonesSkip,0,1,1
...,...,...,...,...
16954,COUNTER,9,1,1
16955,ISSN,9,1,1
16956,00368075back,9,1,1
16957,topterm,9,1,1


### 5. Construire la matrice d'incidence

* Je cré d'abord une matrice de zéro (la largeur prend le nombre de doc + le term, et la longueur sera la nombre de term )

In [68]:
Matrice_incidence = np.zeros((len(np.unique(Dictionnaire['Term'])),len(np.unique(Dictionnaire['Doc n°']))))
Matrice_incidence_df = pd.DataFrame(Matrice_incidence)
Matrice_incidence_df.insert(0, "Term", "")
Matrice_incidence_df

,Term,0,1,2,3,4,5,6,7,8,9
0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11393,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11394,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11395,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11396,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* je remplis ma matrice d'incidence 
* comme j'ai des mots qui sont des espaces (dû au chargement des urls qui font des espaces de tailles différentes), je prend soin de ne prendre que les mots qui ne sont pas des espaces ou tabulation uniquement grace à **.isspace()**
* on met 1 lorsque le mot est dans le doc et 0 sinon 

In [69]:
i=0
list_of_term = []

for term in Dictionnaire['Term']:
    if term.isspace()==False and term not in list_of_term: # si ce n'est pas un espace
        Matrice_incidence_df.at[i,'Term']= term #je met dans la matrice d'incidence le term
        list_of_term.append(term)
        for doc in range(0,10): 
            lign = Dictionnaire.loc[(Dictionnaire['Term']== term) & (Dictionnaire['Doc n°']== doc),:]
            if len(lign) !=0 :
                lign.reset_index(inplace=True, drop=False)

                if lign['nb_fois'][0]!=0:
                    Matrice_incidence_df.at[i,doc]=1
        i += 1 

Matrice_incidence_df

,Term,0,1,2,3,4,5,6,7,8,9
0,COVID,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,research,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,year,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,scientific,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,milestonesSkip,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11393,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11394,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11395,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11396,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* Comme il y avait des mots "espaces", je dois retirer les lignes qui n'ont pas de term pour obtenir ma matrice final

In [70]:
indexMat = Matrice_incidence_df[Matrice_incidence_df['Term']==""].index
Matrice_incidence_df.drop(indexMat , inplace=True)
Matrice_incidence_df


,Term,0,1,2,3,4,5,6,7,8,9
0,COVID,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,research,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,year,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,scientific,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,milestonesSkip,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11377,COUNTER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11378,ISSN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11379,00368075back,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11380,topterm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# A partir d'ici il est possible de tout charger depuis les fichiers : 
* Dictionnaire.pkl --> le dictionnaire créé 
* Matrice_incidence_df.pkl --> la matrice d'incidence créé
* Documents.pkl --> la dataframe des documents complets

In [1]:
# sauvegarder en Dataframe grace à .pkl
#Dictionnaire.to_pickle('Dictionnaire.pkl')  
#Matrice_incidence_df.to_pickle('Matrice_incidence_df.pkl')  
#Pour recharger les dataframe : 
import pandas as pd
Dictionnaire = pd.read_pickle('Dictionnaire.pkl')
Matrice_incidence_df = pd.read_pickle('Matrice_incidence_df.pkl')
#

In [2]:
#filtered_doc=documents.loc[:,documents.columns!="Tokenize_Text"]
#filtered_doc = filtered_doc.loc[:,filtered_doc.columns!="Tokenize_Text_Stop_word"]
#filtered_doc.to_pickle('Documents.pkl')
#écriture de df documents sans les token car non supporté par .pkl
documents = pd.read_pickle('Documents.pkl')


In [3]:
Matrice_incidence_df

,Term,0,1,2,3,4,5,6,7,8,9
0,COVID,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,research,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,year,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,scientific,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,milestonesSkip,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11377,COUNTER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11378,ISSN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11379,00368075back,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11380,topterm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Etape 3 — Construction de l’index inversé
* Après la construction de la matrice d'incidence documents-termes, vous devez construire l'index inversé (liste variable).

In [4]:
index_dictionnaire = {}

for i in range(len(Matrice_incidence_df['Term'])) : 
    doc = []
    for j in range(0,10): 
        if Matrice_incidence_df[j][i] == 1 :
           doc.append(j) 
    index_dictionnaire.update({ Matrice_incidence_df['Term'][i] : doc})

In [5]:
index_dictionnaire # chaque terme, suivi des documents dans lesquels ils apparaissent


{'COVID': [0, 7, 8, 9],
 'research': [0, 2, 8, 9],
 'year': [0, 1, 2, 3, 4, 6, 8, 9],
 'scientific': [0],
 'milestonesSkip': [0],
 'main': [0, 8],
 'contentthank': [0],
 'visit': [0, 1, 2, 3, 4, 6, 7, 8, 9],
 'naturecom': [0],
 '-PRON-': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'use': [0, 1, 2, 3, 4, 6, 7, 8, 9],
 'browser': [0],
 'version': [0, 2, 3, 8, 9],
 'limited': [0, 2, 5, 6, 7, 8],
 'support': [0, 1, 2, 3, 4, 7, 8],
 'css': [0],
 'to': [0, 2, 6, 7, 8, 9],
 'obtainthe': [0],
 'good': [0, 3, 5, 7, 8, 9],
 'experience': [0, 3, 5, 7, 9],
 'recommend': [0, 2, 5, 7],
 'date': [0, 2, 8, 9],
 'turn': [0, 7, 9],
 'compatibility': [0],
 'mode': [0],
 'inInternet': [0],
 'Explorer': [0],
 'in': [0, 1, 2, 3, 4, 5, 7, 8, 9],
 'meantime': [0],
 'ensure': [0, 2, 4, 8],
 'continued': [0, 9],
 'display': [0],
 'site': [0, 1, 2, 3, 6, 7, 8],
 'without': [0, 2, 5, 6, 7, 8, 9],
 'stylesand': [0],
 'JavaScriptAdvertisementView': [0],
 'journalsSearchMy': [0],
 'AccountLoginExplore': [0],
 'contentabout': [

# Etape 4 — Implementation d’un algorithme de recherche d’information  

In [6]:
Dictionnaire.insert(4,'% du texte',"")

* Je choisi de compter le pourcentage de présence de ce mot dans le texte parmis tous les autres mots du texte

In [7]:
nb_word_by_doc = []
for i in range(0,10):
    nb_word_by_doc.append(sum(Dictionnaire[Dictionnaire['Doc n°']==i]["nb_fois"]))

nb_word_by_doc # nombre de mot du dictionnaire (répétition comprise) par document
#la position représente l'ID du doc 

[26859, 2162, 2553, 2576, 2050, 457, 3262, 1688, 9296, 6660]

In [8]:
#ajout du pourcentage de présence du term parmi tous les termes du dictionnaire présents dans le document
for i in range(len(Dictionnaire)) : 
    Dictionnaire.at[i, "% du texte"] = Dictionnaire["nb_fois"][i]/ nb_word_by_doc[Dictionnaire["Doc n°"][i]]*100

Dictionnaire

,Term,Doc n°,nb_fois,nb_de_Doc,% du texte
0,COVID,0,35,4,0.13031
1,research,0,19,4,0.07074
2,year,0,14,8,0.052124
3,scientific,0,2,1,0.007446
4,milestonesSkip,0,1,1,0.003723
...,...,...,...,...,...
16954,COUNTER,9,1,1,0.015015
16955,ISSN,9,1,1,0.015015
16956,00368075back,9,1,1,0.015015
16957,topterm,9,1,1,0.015015


### Début des requêtes

Requêtes booléennes :  
1. desease AND severe  
2. antibody AND plasma AND (cells OR receptors) 
3. antimalarial drugs OR antiviral agents OR immunomodulators 
4. NOT plasma AND risk of infection AND NOT restrictions 
5. (older adults AND antibodies) AND (genomes OR variant)

In [10]:
import en_core_web_sm

* Je définie une fonction pour réaliser mes requêtes 
* Pour le cas "risk of injection", je choisi de chercher dans les docs la chaine de character "risk of injection", car utiliser mon dictionnaire serait éroné
* En effet, les mots étant tokenizés, en faisant une requete qui prend les docs contenant risk et of et injection, n'aurait pas nécessairement ces mots à la suite


In [180]:
nlp = en_core_web_sm.load()

def Request(texte): 
    print("-----", texte, "------")
    res = [texte]
    list_req = nlp(texte)
    
    words = []
    op = []
    result = pd.DataFrame(columns={"Document", "Pourcentage des requêtes"})


    for req in list_req:
        if req.text != "AND" and req.text != "OR" and req.text != "NOT":
            words.append(str(req.lemma_))
        else : 
            op.append(str(req))

            
    # ---- 1 - disease AND severe ----
    if op == ["AND"]: 
        res.append(index_dictionnaire[words[0]] and 
            index_dictionnaire[words[1]]
        )
        for doc in res[1:][0] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)


    #---- 2 - antibody AND plasma AND cells OR receptor ----
    if op == ["AND","AND","OR"] and len(words)==4 : 
        res.append(index_dictionnaire[words[0]] and 
            index_dictionnaire[words[1]] and
            (index_dictionnaire[words[2]] or
            index_dictionnaire[words[3]] )
        )
        for doc in res[1:][0] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[2]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[3]) & (Dictionnaire['Doc n°']== doc),:])

            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)

    

    
    #---- 3 - antimalarial drugs OR antiviral agents OR immunomodulators ----
    if op == ["OR", "OR"]:
        word_str = words[0] + " " + words[1]
        word_str2 = words[2] + " " + words[3]

        for doc in range(0,10):
            if (word_str in documents['Text'][doc]) or (word_str2 in documents['Text'][doc]) or (words[4] in documents['Text'][doc]):
                res.append(doc)
        
        for doc in res[1:] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[2]) & (Dictionnaire['Doc n°']== doc),:])

            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)


    # ---- 4 - NOT plasma AND risk of injection AND NOT restrictions ----
    if op == ["NOT", "AND", "AND", "NOT"] :
        word_str = words[1] + " " + words[2] + " " + words[3]
        for doc in range(0,10):
            if word_str in documents['Text'][doc] and doc not in index_dictionnaire[words[0]] and doc not in index_dictionnaire[words[4]]:
                res.append(doc)

#---- 5 - olders adults AND antibodies AND genomes OR variant ----
    if op == ["AND","AND","OR"] and len(words)!=4:
        word_str = words[0] + " " + words[1]
        for doc in range(0,10):
            if word_str in documents['Text'][doc] and doc in index_dictionnaire[words[2]] and doc in (index_dictionnaire[words[3]] or index_dictionnaire[words[4]]):
                res.append(doc)

        for doc in res[1:] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[2]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[3]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[4]) & (Dictionnaire['Doc n°']== doc),:])

            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)


    

    result = result.sort_values(by=["Pourcentage des requêtes"],ascending=False)
    print(result)


* Lancement des requêtes

In [181]:
request1 = Request("disease AND severe")
request1

----- disease AND severe ------
   Pourcentage des requêtes  Document
1                  1.063830       1.0
5                  0.797057       6.0
6                  0.462565       8.0
2                  0.391696       2.0
7                  0.390390       9.0
4                  0.390244       4.0
0                  0.137756       0.0
3                  0.116460       3.0


In [182]:
request2 = Request("antibody AND plasma AND cells OR receptors")
request2

----- antibody AND plasma AND cells OR receptors ------
   Pourcentage des requêtes  Document
1                  1.397516       3.0
0                  1.046204       0.0
4                  0.398021       8.0
2                  0.341463       4.0
5                  0.225225       9.0
3                  0.030656       6.0


In [183]:
request3 = Request("antimalarial drugs OR antiviral agents OR immunomodulators")
request3

----- antimalarial drugs OR antiviral agents OR immunomodulators ------
   Pourcentage des requêtes  Document
2                  0.048780       4.0
0                  0.003723       0.0
1                  0.000000       2.0


* Pas de résultat pour la requete 4 malgré que je parcours les fichiers pour trouver "risk of injection" (mais je n'ai pas retransformé tout le texte en lemme, il aurait fallu que je fasse des combinaisons de mots à analyser en lemme)

In [184]:
request4 = Request("NOT plasma AND risk of injection AND NOT restrictions")
request4

----- NOT plasma AND risk of injection AND NOT restrictions ------
Empty DataFrame
Columns: [Pourcentage des requêtes, Document]
Index: []


In [185]:
request5 = Request("older adults AND antibodies AND genomes OR variant")
request5

----- older adults AND antibodies AND genomes OR variant ------
Empty DataFrame
Columns: [Pourcentage des requêtes, Document]
Index: []


###  Autres tests complémentaires : 

In [190]:
print(Request("older AND adults"))
print(Request("older AND adults AND antibodies OR disease"))
print(Request("antimalarial AND antiviral"))

----- older AND adults ------
   Pourcentage des requêtes  Document
1                  0.462535       1.0
2                  0.218818       5.0
5                  0.120120       9.0
0                  0.111694       0.0
3                  0.000000       7.0
4                  0.000000       8.0
None
----- older AND adults AND antibodies OR disease ------
   Pourcentage des requêtes  Document
1                  0.462535       1.0
4                  0.146341       4.0
7                  0.120120       9.0
2                  0.117509       2.0
0                  0.111694       0.0
3                  0.038820       3.0
5                  0.000000       6.0
6                  0.000000       8.0
None
----- antimalarial AND antiviral ------
   Pourcentage des requêtes  Document
3                  0.048780       4.0
0                  0.003723       0.0
1                  0.000000       2.0
2                  0.000000       3.0
4                  0.000000       8.0
5                  0.000000 

## Requêtes complexes :  

1. antibody treatments 
2. efficacy and safety of the treatments  
3. family access to hospitals 
4. contact tracing results 
5. genomic analysis of SARS-CoV-2 disease

In [200]:
nlp = en_core_web_sm.load()

def Request_Complexe(texte): 
    print("-----", texte, "------")
    res = [texte]
    list_req = nlp(texte)
    
    words = []
    op = []
    result = pd.DataFrame(columns={"Document", "Pourcentage des requêtes"})


    for req in list_req:
        if req.text != "and" and req.text != "of" and req.text != "the" and req.text != "to":
            words.append(str(req.lemma_))

            
    # ---- antibody treatments 
    if len(words) == 2: 
        res.append(index_dictionnaire[words[0]] and 
            index_dictionnaire[words[1]]
        )
        for doc in res[1:][0] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)


    #---- efficacy and safety of the treatments
    #---- family access to hospitals
    #---- contact tracing results

    if len(words) == 3: 
        res.append(index_dictionnaire[words[0]] and 
            index_dictionnaire[words[1]] and 
            index_dictionnaire[words[2]] 
        )
        for doc in res[1:][0] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[2]) & (Dictionnaire['Doc n°']== doc),:])

            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)


    #---- genomic analysis of SARS-CoV-2 disease
    if len(words) == 4: 
        res.append(index_dictionnaire[words[0]] and 
            index_dictionnaire[words[1]] and 
            (index_dictionnaire[words[2]] or index_dictionnaire['COVID']) and
            index_dictionnaire[words[3]] 

        )
        for doc in res[1:][0] :
            lign = Dictionnaire.loc[(Dictionnaire['Term']== words[0]) & (Dictionnaire['Doc n°']== doc),:]
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[1]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[2]) & (Dictionnaire['Doc n°']== doc),:])
            lign.append(Dictionnaire.loc[(Dictionnaire['Term']== words[3]) & (Dictionnaire['Doc n°']== doc),:])

            result = result.append({"Document":doc,"Pourcentage des requêtes": lign['% du texte'].sum() }, ignore_index=True)

    


    result = result.sort_values(by=["Pourcentage des requêtes"],ascending=False)
    print(result)


In [195]:
Request_Complexe("antibody treatments")

----- antibody treatments ------
   Pourcentage des requêtes  Document
3                  1.397516       3.0
0                  1.046204       0.0
1                  0.601295       1.0
6                  0.398021       8.0
4                  0.341463       4.0
7                  0.225225       9.0
2                  0.078339       2.0
5                  0.030656       6.0


In [196]:
Request_Complexe("efficacy and safety of the treatments")

----- efficacy and safety of the treatments ------
   Pourcentage des requêtes  Document
2                  0.470035       2.0
3                  0.155280       3.0
4                  0.146341       4.0
7                  0.105105       9.0
1                  0.046253       1.0
0                  0.022339       0.0
5                  0.000000       6.0
6                  0.000000       8.0


In [197]:
Request_Complexe("family access to hospitals")

----- family access to hospitals ------
   Pourcentage des requêtes  Document
5                  1.184834       7.0
0                  0.018616       0.0
7                  0.015015       9.0
6                  0.010757       8.0
1                  0.000000       1.0
2                  0.000000       2.0
3                  0.000000       4.0
4                  0.000000       6.0


In [198]:
Request_Complexe("contact tracing results")

----- contact tracing results ------
   Pourcentage des requêtes  Document
0                  0.137756       0.0
8                  0.060060       9.0
7                  0.043029       8.0
2                  0.039170       2.0
6                  0.030656       6.0
1                  0.000000       1.0
3                  0.000000       3.0
4                  0.000000       4.0
5                  0.000000       5.0


* Pas de réponse pour cette request, probablement à cause de SARS-CoV-2 ou alors parce qu'il n'y a pas cette combinaison dans les documents

In [201]:
Request_Complexe('genomic analysis of SARS-CoV-2 disease')

----- genomic analysis of SARS-CoV-2 disease ------
Empty DataFrame
Columns: [Pourcentage des requêtes, Document]
Index: []


In [208]:
word_str = "genomic analysis of SARS-CoV-2 disease"
for doc in range(0,10):
            if (word_str in documents['Text'][doc]):
                print(doc)
            else :
                print("Not in doc ", doc)
        

Not in doc  0
Not in doc  1
Not in doc  2
Not in doc  3
Not in doc  4
Not in doc  5
Not in doc  6
Not in doc  7
Not in doc  8
Not in doc  9


# Conclusion 

* Les n-grams auraient été plus utiles mais je m'en suis rendue compte trop tard et reprendre tout le code serait vraiment trop long
* Cependant pour la plupart des requetes on a les résultats
* Il est également possible d'obtenir les résultats en reformulant les requetes
Par exemple : 


In [212]:
print(Request_Complexe('genomic analysis of COVID'))
print(Request_Complexe('genomic analysis of SARS-CoV-2 disease'))
print(Request_Complexe('genomic analysis of COVID disease'))


----- genomic analysis of COVID ------
   Pourcentage des requêtes  Document
0                  0.029785       0.0
1                  0.000000       7.0
2                  0.000000       8.0
3                  0.000000       9.0
None
----- genomic analysis of SARS-CoV-2 disease ------
Empty DataFrame
Columns: [Pourcentage des requêtes, Document]
Index: []
None
----- genomic analysis of COVID disease ------
   Pourcentage des requêtes  Document
0                  0.029785       0.0
1                  0.000000       1.0
2                  0.000000       2.0
3                  0.000000       3.0
4                  0.000000       4.0
5                  0.000000       5.0
6                  0.000000       6.0
7                  0.000000       7.0
8                  0.000000       8.0
9                  0.000000       9.0
None
